In [2]:
import os
import cv2
import pandas as pd
from utils import *
from gNet import GestureNet
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.client import device_lib
from keras import Sequential, Input
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
from datetime import datetime
from sklearn.model_selection import train_test_split
import pandas as pd

## GPU available test

In [3]:
print(device_lib.list_local_devices())
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14840139673341481497
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1297474971
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17071686200178179586
physical_device_desc: "device: 0, name: NVIDIA GeForce MX450, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


## Define Hyperparameter and build model

In [17]:

batch_size = 4
epoch = 100

backup_dir = Path('./model/') / "primary"
ckpt_location = backup_dir / "model_ckpt.ckpt"
model_location = backup_dir


ckpt_backup_callback = tf.keras.callbacks.ModelCheckpoint(filepath=ckpt_location, save_weights_only=True, verbose=1)

label = getLabel()
X, Y = getTrain2DF()
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, stratify=Y)
gnet = GestureNet()
model = gnet.model()
model.summary()

Model: "GestureNet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 48)                2064      
                                                                 
 batch_normalization_6 (Batc  (None, 48)               192       
 hNormalization)                                                 
                                                                 
 activation_6 (Activation)   (None, 48)                0         
                                                                 
 dropout_6 (Dropout)         (None, 48)                0         
                                                                 
 dense_10 (Dense)            (None, 24)                1176      
                                                                 
 batch_normalization_7 (Batc  (None, 24)               96        
 hNormalization)                                        

## Train model

In [18]:
model.fit(x_train, y_train, epochs=epoch, callbacks=[ckpt_backup_callback])
model.save(model_location / "model.h5")

Epoch 1/100
153/155 [============================>.] - ETA: 0s - loss: 0.8965 - accuracy: 0.7255
Epoch 1: saving model to model\primary\model_ckpt.ckpt
155/155 [==============================] - 2s 6ms/step - loss: 0.8942 - accuracy: 0.7263
Epoch 2/100
154/155 [============================>.] - ETA: 0s - loss: 0.4521 - accuracy: 0.8815
Epoch 2: saving model to model\primary\model_ckpt.ckpt
155/155 [==============================] - 1s 6ms/step - loss: 0.4517 - accuracy: 0.8817
Epoch 3/100
154/155 [============================>.] - ETA: 0s - loss: 0.3440 - accuracy: 0.9048
Epoch 3: saving model to model\primary\model_ckpt.ckpt
155/155 [==============================] - 1s 5ms/step - loss: 0.3441 - accuracy: 0.9047
Epoch 4/100
147/155 [===========================>..] - ETA: 0s - loss: 0.2806 - accuracy: 0.9145
Epoch 4: saving model to model\primary\model_ckpt.ckpt
155/155 [==============================] - 1s 5ms/step - loss: 0.2807 - accuracy: 0.9157
Epoch 5/100
153/155 [===============

## Load model

In [20]:
# model.load_weights(ckpt_location)
model = keras.models.load_model(model_location / "model.h5")
loss, acc = model.evaluate(x_test, y_test, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

39/39 - 0s - loss: 0.0158 - accuracy: 0.9960 - 212ms/epoch - 5ms/step
Restored model, accuracy: 99.60%


In [21]:
law = [0.0, 0.0, -0.08523409363745499, 0.004801920768307323, -0.16566626650660263, -0.027611044417767107, -0.21728691476590636, -0.062424969987995196, -0.2665066026410564, -0.08883553421368548, -0.15966386554621848, -0.16326530612244897, -0.22448979591836735, -0.22809123649459784, -0.2689075630252101, -0.2653061224489796, -0.3097238895558223, -0.29411764705882354, -0.1092436974789916, -0.19447779111644659, -0.15846338535414164, -0.2809123649459784, -0.1908763505402161, -0.33733493397358943, -0.22088835534213686, -0.3865546218487395, -0.05042016806722689, -0.20048019207683074, -0.08283313325330131, -0.2917166866746699, -0.09963985594237695, -0.3517406962785114, -0.11764705882352941, -0.4069627851140456, 0.013205282112845138, -0.18847539015606243, 0.004801920768307323, -0.2581032412965186, -0.003601440576230492, -0.304921968787515, -0.015606242496998799, -0.34933973589435774]
df = pd.DataFrame(law).T
print(df.shape)
pred = model.predict(df)

print(pred)


(1, 42)
1/1 [==============================] - 0s 64ms/step
[[8.7999057e-09 1.9449234e-07 9.9999821e-01 1.6870815e-09 2.4801021e-09
  1.5642159e-06]]
